In [ ]:
%pip install --upgrade setuptools
%pip install --upgrade gradio

In [ ]:
import json
import setuptools

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from dotenv import dotenv_values
from typechat import Failure, TypeChatJsonTranslator, TypeChatValidator, create_language_model
from examples.restaurant import schema as restaurant

In [ ]:
env_vals = dotenv_values()
model = create_language_model(env_vals)
validator = TypeChatValidator(restaurant.Order)
translator = TypeChatJsonTranslator(model, validator, restaurant.Order)

In [ ]:
import pandas
async def get_translation(message, history):
    result = await translator.translate(message)
    if isinstance(result, Failure):
        return f"Translation Failed ❌ \n Context: {result.message}"
    else:
        result = result.value
        df = pandas.DataFrame.from_dict(result["items"])
        return f"Translation Succeeded! ✅\n Restaurant orders \n ``` {df.fillna('').to_markdown(tablefmt='grid')} \n ```  \n"


def get_examples():
    example_prompts = []
    with open('../examples/restaurant/input.txt') as prompts_file:
        for line in prompts_file:
            example_prompts.append(line)
    return example_prompts


In [ ]:
import gradio as gr

gr.ChatInterface(get_translation, title="🍕 Restaurant", examples=get_examples()).launch()
